In [1]:
!pip install gdown
import gdown
url = 'https://drive.google.com/uc?id=1_WuUfkGq9z_WKBTw-S1mjGlk4IhkQwM0' 
output = './JPG_FILE.tar' 
gdown.download(url, output, quiet=False)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.13.0-py3-none-any.whl size=9034 sha256=9cd54d01495a26508e25b40f41237bfb02aedac93e5733571a00550e1232482e
  Stored in directory: /root/.cache/pip/wheels/2f/2a/2f/86449b6bdbaa9aef873f68332b68be6bfbc386b9219f47157d
Successfully built gdown


Downloading...
From: https://drive.google.com/uc?id=1_WuUfkGq9z_WKBTw-S1mjGlk4IhkQwM0
To: /kaggle/working/JPG_FILE.tar
788MB [00:06, 115MB/s]  


'./JPG_FILE.tar'

In [2]:
import tarfile

#simple function to extract the train data
#tar_file : the path to the .tar file
#path : the path where it will be extracted
def extract(tar_file, path):
    opened_tar = tarfile.open(tar_file)
     
    if tarfile.is_tarfile(tar_file):
        opened_tar.extractall(path)
    else:
        print("The tar file you entered is not a tar file")
extract('./JPG_FILE.tar','./PovertyCelebes/')

In [3]:
!pip install fastai --upgrade -q
import fastai
from fastai import *
from fastai.vision import *
from fastai.vision.core import *
from fastai.vision.data import *
from fastai.vision.augment import *
from fastai.vision.all import *
from fastai.text.all import *
from fastai.collab import *
from fastai.tabular.all import *
from fastai.callback.all import *
from fastai.metrics import error_rate,accuracy
import gc
import pandas as pd
import re
import os
import seaborn as sbs

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kornia 0.5.5 requires numpy<=1.19, but you have numpy 1.19.5 which is incompatible.
allennlp 2.5.0 requires torch<1.9.0,>=1.6.0, but you have torch 1.9.0 which is incompatible.
allennlp 2.5.0 requires torchvision<0.10.0,>=0.8.1, but you have torchvision 0.10.0 which is incompatible.


In [4]:
learn = load_learner('../input/povertycelebes-model/CNN_LRNR_RES34_IND_2020_LS_256_new_fast_propert (2).pkl')
learn.predict('./PovertyCelebes/content/drive/MyDrive/POVERTY_CELEBES_CNN_IMGB_IND_2015_LS_256_TIF_3840/JPG_FILE/CNN_DIMG_IND_2015_LS_256_3840_000001_REV.jpg')

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


('3', tensor(3), tensor([0.0575, 0.3669, 0.0807, 0.4778, 0.0172]))

In [5]:
learn.model[1][6]

BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [22]:
class Hook():
    def hook_func(self, m, i, o): self.stored = o.detach().clone()
hook_output = Hook()
hook = learn.model[1][6].register_forward_hook(hook_output.hook_func)

In [34]:
df=pd.read_csv('../input/bin-gmm/bin_cluster_GMM.csv',sep=';').reset_index()
df.columns
df['index']=df['index'].apply(lambda y:str(y).zfill(6))

In [45]:
features_out = np.empty((len(df["index"]), 512))
features_out_img = df["GRID_SAT_JPG"]

In [46]:
root_folder_='./PovertyCelebes/content/drive/MyDrive/POVERTY_CELEBES_CNN_IMGB_IND_2015_LS_256_TIF_3840/JPG_FILE/'
for i, path_i in enumerate(features_out_img):
    if i%100==0:
        print(i/100,end='-')
    temp_img =root_folder_+path_i
    with learn.no_bar():
        temp_pred=learn.predict(temp_img)
        tempfeatures = hook_output.stored[0].flatten().reshape(1,-1).numpy()
        features_out[i,:] = tempfeatures

0.0-1.0-2.0-3.0-4.0-5.0-6.0-7.0-8.0-9.0-10.0-11.0-12.0-13.0-14.0-15.0-16.0-17.0-18.0-19.0-20.0-21.0-22.0-23.0-24.0-25.0-26.0-27.0-28.0-29.0-30.0-31.0-32.0-33.0-34.0-35.0-36.0-37.0-38.0-39.0-40.0-41.0-42.0-43.0-44.0-45.0-46.0-47.0-48.0-49.0-50.0-51.0-52.0-53.0-54.0-55.0-56.0-57.0-58.0-59.0-60.0-61.0-62.0-63.0-64.0-65.0-66.0-67.0-68.0-69.0-70.0-71.0-72.0-73.0-74.0-75.0-76.0-77.0-78.0-79.0-80.0-81.0-82.0-83.0-84.0-85.0-86.0-87.0-88.0-89.0-90.0-91.0-92.0-93.0-94.0-95.0-96.0-97.0-98.0-99.0-100.0-101.0-102.0-103.0-104.0-105.0-106.0-107.0-108.0-109.0-110.0-111.0-112.0-113.0-114.0-115.0-116.0-117.0-118.0-119.0-120.0-121.0-122.0-123.0-124.0-125.0-126.0-

In [47]:
features_out_pd = pd.DataFrame(data = features_out, index = features_out_img)
features_out_pd.to_csv('./feature_extracted.csv',sep=';')

In [48]:
features_out_pd

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
GRID_SAT_JPG,,,,,,,,,,,,,,,,,,,,,
CNN_DIMG_IND_2015_LS_256_3840_000001_REV.jpg,1.483518,-0.549518,-0.618008,0.134535,-0.665146,-0.536729,-0.504519,-0.553781,-0.558263,-0.354462,...,-0.374221,1.625100,-0.611686,2.457668,-0.571589,-0.252379,-0.021947,-0.469171,-0.542241,-0.251967
CNN_DIMG_IND_2015_LS_256_3840_000002_REV.jpg,2.838306,0.430732,-0.618008,-0.504173,-0.665146,-0.536729,-0.159349,-0.553781,-0.640302,1.601868,...,-0.637575,0.746244,-0.611686,4.535200,-0.571589,1.162027,1.007501,-0.196767,-0.341298,-0.126327
CNN_DIMG_IND_2015_LS_256_3840_000003_REV.jpg,1.483518,-0.549518,-0.618008,0.134535,-0.665146,-0.536729,-0.504519,-0.553781,-0.558263,-0.354462,...,-0.374221,1.625100,-0.611686,2.457668,-0.571589,-0.252379,-0.021947,-0.469171,-0.542241,-0.251967
CNN_DIMG_IND_2015_LS_256_3840_000004_REV.jpg,-0.494105,1.132504,-0.618008,0.529057,-0.560871,-0.536729,-0.504519,-0.553781,-0.640302,-0.304400,...,-0.068745,0.652662,-0.122190,-0.430145,-0.571589,-0.051411,-0.579124,-0.469171,0.513522,0.170990
CNN_DIMG_IND_2015_LS_256_3840_000005_REV.jpg,2.260132,1.686836,-0.618008,-0.504173,-0.610025,-0.536729,-0.504519,-0.553781,-0.171594,0.282005,...,0.215796,1.484893,0.606741,-0.121295,-0.571589,-0.461053,0.140711,-0.469171,-0.174309,0.520053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CNN_DIMG_IND_2015_LS_256_3840_012686_REV.jpg,1.993460,0.378919,-0.618008,-0.504173,-0.665146,-0.536729,-0.504519,-0.553781,-0.640302,2.069283,...,-0.499233,1.944110,-0.611686,1.951528,-0.571589,0.651559,-0.172969,-0.148752,-0.555317,-0.397953
CNN_DIMG_IND_2015_LS_256_3840_012687_REV.jpg,3.008845,-0.102012,-0.618008,-0.504173,-0.665146,-0.536729,-0.285589,-0.553781,-0.590720,1.867534,...,-0.637575,0.985295,-0.611686,3.052529,-0.571589,-0.461053,-0.579124,-0.415597,-0.555317,0.609186
CNN_DIMG_IND_2015_LS_256_3840_012688_REV.jpg,0.721480,1.494075,-0.618008,-0.504173,-0.665146,-0.536729,-0.504519,-0.553781,-0.640302,1.893263,...,-0.637575,0.920031,-0.611686,2.572822,-0.571589,1.327093,0.299578,0.025175,-0.555317,-0.001615
